# Selección de características en modelos predictivos

Este proyecto es para IA de Moisés Calzado Cobo y Antonio Germán Márquez Trujillo

Estos archivos contienen los algoritmos implementados, pero no los importamos ya que en esta libreta los implementamos explicando su funcionamiento


In [62]:
import pandas as pandas;
import time;
from sklearn.model_selection import cross_val_score;
from sklearn import tree;
from funcy import join
from sklearn import preprocessing;
import numpy as np;
import matplotlib.pyplot as plt
pandas.set_option('max_colwidth', 800)

In [2]:
datos = pandas.read_csv("datos/titanic.csv")
#datos = pandas.read_csv("datos/BreastCancerDataset.csv")
datos.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Initial,Age_band,Family_Size,Alone,Fare_cat,Deck,Title,Is_Married,Survived
0,3,0,22.0,1,0,7.2500,0,0,1,1,0,0,7,11,0,0
1,1,1,38.0,1,0,71.2833,1,1,2,1,0,3,2,12,1,1
2,3,1,26.0,0,0,7.9250,0,2,1,0,1,1,7,8,0,1
3,1,1,35.0,1,0,53.1000,0,1,2,1,0,3,2,12,1,1
4,3,0,35.0,0,0,8.0500,0,0,2,0,1,1,7,11,0,0


In [55]:
def evaluar_soluciones(datos, solucion_actual, nueva_variable, objetivo, n_exp, cv):
    variables = solucion_actual[:]
    variables.append(nueva_variable)
    data_frame = pandas.DataFrame(data=datos)
    X = data_frame[variables]
    y = data_frame[objetivo]
    clf = tree.DecisionTreeClassifier()
    
    scores = cross_val_score(clf, X, y, cv=cv, scoring="balanced_accuracy")
    
    for i in range(n_exp-1):
        new_scores = cross_val_score(clf, X, y, cv=cv, scoring="balanced_accuracy") 
        scores = scores + new_scores
    
    scores = scores/n_exp
    clave = ', '.join(variables)
    
    diccionario_resultado = {}
    diccionario_resultado[clave] = np.mean(scores)
    return diccionario_resultado

In [63]:
import multiprocessing as mp
def SFS(datos, respuesta, d = 0):
    start = time.time()

    diccionario_resultado = {}
    columnas = list(datos.columns)
    columnas.remove(respuesta)
    solucion_actual = []

    k = 0
    d = d if d else len(columnas)
    
    while(k<d):
        pool = mp.Pool(mp.cpu_count())
        new_resultados = pool.starmap(evaluar_soluciones, [(datos, solucion_actual, nuevaVariable, respuesta, 15, 10) for nuevaVariable in columnas])
        pool.close()
        resultado = join(new_resultados)
        
        variable_escogida = max(resultado, key=resultado.get)
        variables_nuevas = variable_escogida.split(", ")
        solucion_actual.append(variables_nuevas[len(variables_nuevas)-1])
        columnas.remove(variables_nuevas[len(variables_nuevas)-1])

        k = k+1
        
        diccionario_resultado[variable_escogida] = resultado[variable_escogida]
    
    done = time.time()
    elapsed = done - start
    print(elapsed)
    return diccionario_resultado

In [64]:
resultado_SFS = SFS(datos, "Survived", 3)
print(resultado_SFS)

2.394841432571411
{'Initial': 0.7833541295306, 'Initial, SibSp': 0.8052182893359363, 'Initial, SibSp, Deck': 0.8077201991907874}
